In [1]:
import numpy as np
import pandas as pd
import pyodbc
import json
from collections import defaultdict

In [2]:
server = "192.168.10.35"
database = "MarketDB"
username = "mostafa"
password = "mostafa1234"
driver = "ODBC Driver 18 for SQL Server"

conn_str = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Encrypt=yes;"
    f"TrustServerCertificate=yes;"
)

conn = pyodbc.connect(conn_str)

query = """SELECT [StockID]
      ,[Name]
      ,[FullName]
  FROM [MarketDB].[dbo].[SmallStocks]
"""

stocks = pd.read_sql_query(query, conn)

C:\Users\Mesbah\AppData\Local\Temp\ipykernel_6872\3503057335.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stocks = pd.read_sql_query(query, conn)


In [3]:
stocks

,StockID,Name,FullName
0,114312662654155,وسپهر,سرمایه گذاری مالی سپهرصادرات
1,118005828419984,چتر,صندوق س.بخشی گستره فیروزه2 ب
2,204092872752957,شصدف,صنعتی دوده فام
3,318005355896147,فسا,پتروشیمی فسا
4,383529400851043,الکتروماد,صنایع الکترونیک مادیران
...,...,...,...
1128,71856634742001725,سپ,پرداخت الکترونیک سامان کیش
1129,71907403347534074,غزنجان,شیر پاستوریزه پگاه زنجان
1130,71945594172117613,ارزش مسکن,ص.س.املاک ومستغلات ارزش مسکن
1131,71957984642204570,شپترو,پتروشیمی آبادان


In [ ]:
server = "192.168.10.35"
database = "OrderbookHistoryDb"
username = "mostafa"
password = "mostafa1234"
driver = "ODBC Driver 18 for SQL Server"

conn_str = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Encrypt=yes;"
    f"TrustServerCertificate=yes;"
)

conn = pyodbc.connect(conn_str)

df = pd.DataFrame([])

for stock in stocks.iloc[: , 0]:
    query = f"""SELECT TOP (300) [StockID]
        ,[Date]
        ,[orders]
    FROM [OrderbookHistoryDb].[dbo].[NewOrderBooks] 
    where StockID = {stock}
    order by [Date] desc 
    """
    df = pd.concat([df , pd.read_sql_query(query, conn)] , axis=0)
    # df = pd.read_sql_query(query, conn)
    
df.to_csv("orders.csv", index=False, encoding="utf-8-sig")

C:\Users\Mesbah\AppData\Local\Temp\ipykernel_6872\815315218.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.concat([df , pd.read_sql_query(query, conn)] , axis=0)
C:\Users\Mesbah\AppData\Local\Temp\ipykernel_6872\815315218.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.concat([df , pd.read_sql_query(query, conn)] , axis=0)
C:\Users\Mesbah\AppData\Local\Temp\ipykernel_6872\815315218.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.concat([df , pd.read_sql_query(query, conn)] , axis=0)
C

In [ ]:
df = pd.read_csv("orders.csv")

In [8]:
df

,StockID,Date,orders
0,114312662654155,2025-10-22,"[[60124, 1, 1, 1, 3392, 10000], [60124, 1, 2, ..."
1,114312662654155,2025-10-21,"[[60125, 1, 1, 1, 3392, 10000], [60125, 1, 2, ..."
2,114312662654155,2025-10-20,"[[60122, 1, 1, 1, 3442, 8000], [60122, 1, 2, 1..."
3,114312662654155,2025-10-19,"[[60124, 1, 1, 1, 3450, 5000], [60124, 1, 2, 1..."
4,114312662654155,2025-10-18,"[[60125, 1, 1, 1, 3549, 30000], [60125, 1, 2, ..."
...,...,...,...
290998,72044846109864381,2023-12-30,"[[60129, 1, 1, 1, 6830, 13298], [60129, 1, 2, ..."
290999,72044846109864381,2023-12-27,"[[60128, 1, 1, 1, 6830, 13298], [60128, 1, 2, ..."
291000,72044846109864381,2023-12-26,"[[60127, 1, 1, 1, 6830, 13298], [60127, 1, 2, ..."
291001,72044846109864381,2023-12-25,"[[60128, 1, 1, 2, 7190, 16088], [60128, 1, 2, ..."


In [5]:
for index, row in df.iterrows():
    orders = json.loads(row["orders"])
    filtered_orders = [i for i in orders if i[1] == 1]
    
    df.at[index, 'orders'] = json.dumps(filtered_orders, ensure_ascii=False)

In [6]:
df.head()

,StockID,Date,orders
0,114312662654155,2025-10-22,"[[60124, 1, 1, 1, 3392, 10000], [60124, 1, 2, ..."
1,114312662654155,2025-10-21,"[[60125, 1, 1, 1, 3392, 10000], [60125, 1, 2, ..."
2,114312662654155,2025-10-20,"[[60122, 1, 1, 1, 3442, 8000], [60122, 1, 2, 1..."
3,114312662654155,2025-10-19,"[[60124, 1, 1, 1, 3450, 5000], [60124, 1, 2, 1..."
4,114312662654155,2025-10-18,"[[60125, 1, 1, 1, 3549, 30000], [60125, 1, 2, ..."


In [ ]:
def filtered_handler(order_list):

    data = json.loads(order_list)

    results = []

    filtered_data = [
        row
        for row in data
        if 90000 <= (row[0][0] if isinstance(row[0], list) else row[0]) <= 123000
    ]

    groups = {}
    for item in filtered_data:
        key = item[0][0] if isinstance(item[0], list) else item[0]
        groups.setdefault(key, []).append(item)

    for _, items in groups.items():

        for i in range(len(items)):
            for j in range(i + 1, len(items)):
                a, b = items[i], items[j]

                a_id = a[0][0] if isinstance(a[0], list) else a[0]
                b_id = b[0][0] if isinstance(b[0], list) else b[0]

                if not (90000 <= a_id <= 123000):
                    continue
                if not (90000 <= b_id <= 123000):
                    continue

                if a[2] != b[2]:
                    a_zero = all(x == 0 for x in a[3:6])
                    b_zero = all(x == 0 for x in b[3:6])

                    if a_zero != b_zero:
                        zero_item = a if a_zero else b
                        target_c3 = zero_item[2]

                        candidate = None
                        for idx, t in enumerate(filtered_data):
                            if t[2] == target_c3 and all(x != 0 for x in t[3:6]):
                                t_id = t[0][0] if isinstance(t[0], list) else t[0]
                                new_val = t_id - 20

                                for back_idx in range(idx, -1, -1):
                                    k = filtered_data[back_idx]
                                    k_id = k[0][0] if isinstance(k[0], list) else k[0]
                                    if k_id <= new_val:
                                        candidate = k
                                        break
                                break

                        if candidate and candidate not in results:
                            results.append(candidate)

    # print("نتیجه:", results)

    return json.dumps(results)


df["OrderQueue"] = df["orders"].apply(filtered_handler)

In [17]:
df.sample(10)

,StockID,Date,OrderQueue
27025,6757220448540984,2025-04-26,"[[90140, 1, 1, 5, 30290, 8893]]"
194687,48261930411425125,2024-11-30,[]
106035,27000326841257664,2024-12-01,[]
131650,32941062994509455,2025-07-19,[]
172205,43283802997035462,2024-09-15,[]
127685,31879190587976736,2024-06-16,[]
100221,25244329144808274,2024-12-14,[]
247828,62346804681275278,2025-10-18,"[[103002, 1, 1, 3, 31100, 3160]]"
234910,59142194115401696,2024-07-22,[]
69358,17914401175772326,2025-01-26,[]


In [13]:
df.drop(["orders"] , axis=1 , inplace=True)

In [18]:
df.to_csv("order_result.csv" ,  index=False, encoding="utf-8-sig")

In [ ]:
from sqlalchemy import create_engine
import urllib.parse

database = "Boors"
server = "njweb"
username = "Boors"
password = urllib.parse.quote_plus("Boors2025@")
driver = "ODBC+Driver+18+for+SQL+Server"

connection_string = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    f"?driver={driver}"
    "&Encrypt=no"
    "&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)

with engine.connect() as conn:
    df.to_sql("OrderBook", con=conn, if_exists="replace", index=False)